In [3]:
from dictionary_learning import CrossCoder
from torch.nn.functional import cosine_similarity
import torch as th
import plotly.express as px
from pathlib import Path
from tqdm.notebook import tqdm
import pandas as pd

th.set_grad_enabled(False)
exp_name = "eval_crosscoder"
# %load_ext autoreload
# %autoreload 2

In [4]:
data = th.load("/dlabscratch1/jminder/repositories/representation-structure-comparison/high_threshold_indices.pt")

/tmp/ipykernel_1173961/77499533.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = th.load("/dlabscratch1/jminder/repositories/representation-structure-comparison/hi

In [5]:
!export TOKENIZERS_PARALLELISM=false

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
import sys
sys.path.append("..")
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [7]:
crosscoder_path = "/dlabscratch1/jminder/repositories/representation-structure-comparison/checkpoints/l13-mu4.1e-02-lr1e-04/ae_final.pt"
extra_args = []
exp_id = ""
device = "cuda"
seed = 42
base_model = "google/gemma-2-2b"
instruct_model = "google/gemma-2-2b-it"
layer = 13
activation_dir = Path(
    "/dlabscratch1/jminder/repositories/representation-structure-comparison/activations"
)
validation_size = 10**6
model_batch_size = 64
workers = 12
SEQ_LEN = 1024
n = 100
crosscoder_batch_size = 2048

In [19]:
data_path = Path(
    "/dlabscratch1/cdumas/representation-structure-comparison/notebooks/results/eval_crosscoder/l13-mu4.1e-02-lr1e-04_ae_final/data"
)
df = pd.read_csv(data_path / "feature_df.csv")
# Filter for IT only and Base only features that are not dead
selected_features = df[(df['tag'].isin(['IT only', 'Base only'])) & (df['dead'] == False)]

# Get the indices of the selected features in the dataframe
selected_indices = selected_features.index.tolist()

print(f"Number of selected features: {len(selected_indices)}")
print(f"First 10 selected indices: {selected_indices[:10]}")
# display top 70 features order by dec_norm_diff ascending
sorted_df = df.sort_values(by='dec_norm_diff', ascending=True)
display(sorted_df.head(70))
# plot rank of dead_scale_features_1e_4 features in this df in an histogram using plotly
ft_sorted = sorted_df.index.tolist()
print(ft_sorted[:10])
ranks = [ft_sorted.index(i) for i in dead_scale_features_1e_4]
import plotly.express as px
import numpy as np

# Calculate the fraction of dead features with rank < x
x = np.arange(0, max(ranks) + 1)
y = [sum(r <= i for r in ranks)/len(ranks) for i in x]

# Create a line plot
fig = px.line(x=x, y=y, title="Fraction of dead_scale_features_1e_4 features with rank < x")
fig.update_layout(xaxis_title="Rank", yaxis_title="Fraction of dead features")
fig.show()

Number of selected features: 4532
First 10 selected indices: [55, 60, 78, 82, 95, 112, 119, 130, 140, 221]


,Unnamed: 0,fw_dead,fw_freq,lmsys_dead,lmsys_freq,freq,dead,avg_activation,lmsys_avg_activation,fw_avg_activation,dec_base_norm,dec_instruct_norm,dec_norm_diff,tag,enc_base_norm,enc_instruct_norm,enc_norm_diff,dec_cos_sim,enc_cos_sim
46634,46634,False,0.000787,False,0.002868,0.001720,False,29.239553,30.171347,26.476154,0.001748,0.338830,0.002580,IT only,0.417732,0.789705,0.264486,0.477075,-0.043931
22195,22195,False,0.000811,False,0.002958,0.001774,False,27.935972,28.831285,25.280428,0.001749,0.338141,0.002587,IT only,0.416474,0.783407,0.265809,0.476522,-0.044766
34491,34491,False,0.000820,False,0.002807,0.001711,False,26.260895,27.544521,22.687355,0.001754,0.334975,0.002618,IT only,0.393317,0.737529,0.266645,0.475791,-0.040318
40524,40524,False,0.000767,False,0.002750,0.001657,False,26.192369,27.079718,23.606073,0.001757,0.334098,0.002630,IT only,0.393061,0.719624,0.273102,0.478361,-0.018557
24336,24336,False,0.000796,False,0.002943,0.001759,False,22.434280,23.219576,20.073692,0.001747,0.331629,0.002633,IT only,0.387836,0.725761,0.267193,0.469173,-0.023145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46111,46111,False,0.002643,False,0.000806,0.001819,False,2.707451,2.462946,2.768109,0.018679,0.789474,0.011830,IT only,0.659694,1.343311,0.245548,0.358147,-0.188594
73524,73524,True,0.000000,True,0.000000,0.000000,True,NaN,NaN,NaN,0.001944,0.081587,0.011913,IT only,0.163101,0.206286,0.395327,0.168691,0.175323
62733,62733,False,0.000525,False,0.002459,0.001393,False,3.647012,4.005270,2.283152,0.020090,0.824929,0.012177,IT only,0.930646,1.634067,0.284764,0.427975,-0.594102
51224,51224,False,0.002244,False,0.009417,0.005461,False,1.356139,1.434295,1.089368,0.015165,0.620764,0.012215,IT only,0.781824,1.287962,0.303512,0.678796,-0.315993


[46634, 22195, 34491, 40524, 24336, 20970, 35868, 17371, 1547, 53334]


## Feature identification

In [ ]:
base_only_features = df[df['tag'] == 'Base only'].sort_values(by='lmsys_freq', ascending=False).head(50)
base_only_features

In [8]:
df.iloc[593]

Unnamed: 0                    593
fw_dead                     False
fw_freq                  0.001531
lmsys_dead                  False
lmsys_freq               0.001429
freq                     0.001485
dead                        False
avg_activation            3.63542
lmsys_avg_activation     3.992857
fw_avg_activation        3.364052
dec_base_norm            0.700494
dec_instruct_norm        0.027105
dec_norm_diff            0.980653
tag                     Base only
enc_base_norm            1.852539
enc_instruct_norm        0.606914
enc_norm_diff            0.836194
dec_cos_sim              0.674656
enc_cos_sim              -0.07113
Name: 593, dtype: object

In [13]:
import json
it_only_features = th.load(data_path / "only_it_decoder_feature_indices.pt", weights_only=True)
dead_scale_features_1e_4 = json.load(
    open(save_path.parent / "scalar" / "dead_feature_indices_1e-4.json", "r")
)
dead_scale_features_1e_4 = [it_only_features[i].item() for i in dead_scale_features_1e_4]
print(len(dead_scale_features_1e_4))
print(f"not deads: {len([i for i in dead_scale_features_1e_4 if not df.iloc[i]['dead']])}")
dead_scale_features_all = json.load(
    open(save_path.parent / "scalar" / "dead_feature_indices_1e-2.json", "r")
)
dead_scale_features_all = [it_only_features[i] for i in dead_scale_features_all]
with pd.option_context('display.max_rows', None):
    display(df[df.index.isin(dead_scale_features_1e_4) & (df['dead'] == False)].style.format({'lmsys_freq': '{:.1e}', 'fw_freq': '{:.1e}', 'freq': '{:.1e}'})) # .sort_values(by='lmsys_freq', ascending=False)

70
not deads: 70


,Unnamed: 0,fw_dead,fw_freq,lmsys_dead,lmsys_freq,freq,dead,avg_activation,lmsys_avg_activation,fw_avg_activation,dec_base_norm,dec_instruct_norm,dec_norm_diff,tag,enc_base_norm,enc_instruct_norm,enc_norm_diff,dec_cos_sim,enc_cos_sim
771,771,False,6.2e-03,False,1.1e-03,3.9e-03,False,1.909597,1.950318,1.903743,0.019161,0.601516,0.015928,IT only,0.932539,1.535208,0.303717,0.518771,-0.523397
883,883,False,2.5e-03,False,3.6e-03,3.0e-03,False,3.717293,3.935713,3.455262,0.016195,0.695830,0.011637,IT only,0.911130,1.705567,0.267105,0.612343,-0.028956
1045,1045,False,8.2e-04,False,3.4e-03,2.0e-03,False,2.844598,3.318568,1.259410,0.036045,0.798618,0.022567,IT only,1.015253,1.621810,0.313000,0.547398,-0.626637
1353,1353,False,3.1e-04,False,7.2e-04,5.0e-04,False,5.033971,5.494558,4.174287,0.051135,0.809860,0.031570,IT only,0.951458,1.758868,0.270474,0.509238,-0.093958
1742,1742,False,2.2e-03,False,9.6e-04,1.7e-03,False,2.415570,2.296070,2.457325,0.035891,0.584446,0.030705,IT only,0.849956,1.651648,0.257305,0.677590,-0.223040
2412,2412,False,2.0e-03,False,1.5e-03,1.8e-03,False,3.363636,3.214202,3.454763,0.031885,0.653488,0.024396,IT only,0.849247,1.657459,0.256189,0.697360,-0.092483
3129,3129,False,5.6e-03,False,3.5e-03,4.7e-03,False,2.808331,2.736047,2.845446,0.012027,0.638428,0.009419,IT only,0.873279,1.563975,0.279186,0.444073,-0.273691
3857,3857,False,3.7e-03,False,1.2e-03,2.6e-03,False,2.698874,2.617466,2.720279,0.022227,0.664758,0.016718,IT only,0.825620,1.507581,0.273823,0.491178,-0.380666
4194,4194,False,1.8e-03,False,1.2e-03,1.6e-03,False,2.712537,2.548630,2.801038,0.019319,0.659032,0.014657,IT only,0.824485,1.631766,0.252636,0.376209,-0.042035
5406,5406,False,8.8e-05,False,3.7e-04,2.1e-04,False,7.158593,7.868626,4.727653,0.112356,0.940421,0.059737,IT only,1.088786,1.564219,0.348029,0.230806,-0.107499


In [ ]:
df[df.index.isin(dead_scale_features_all)] # .sort_values(by='lmsys_freq', ascending=False)

In [21]:
import json
fossil_features = json.load(open("/dlabscratch1/cdumas/representation-structure-comparison/notebooks/results/eval_crosscoder/l13-mu4.1e-02-lr1e-04_ae_final/data/lmsys_it_fossil_indices.json", "r"))
print(fossil_features)
# show df for fossil features
df[df.index.isin(fossil_features)].sort_values(by='lmsys_freq', ascending=False)


[1697, 1746, 2667, 3152, 3792, 3913, 4179, 4874, 4940, 6911, 7269, 11743, 12121, 14538, 21710, 24731, 27129, 28200, 29706, 30133, 30142, 30859, 34253, 35465, 38917, 39351, 44612, 45142, 50764, 51124, 51224, 51492, 54708, 55240, 55641, 56484, 56981, 57713, 59641, 60186, 61285, 62571, 63664, 67357, 69536]


,Unnamed: 0,fw_dead,fw_freq,lmsys_dead,lmsys_freq,freq,dead,avg_activation,lmsys_avg_activation,fw_avg_activation,dec_base_norm,dec_instruct_norm,dec_norm_diff,tag,enc_base_norm,enc_instruct_norm,enc_norm_diff,dec_cos_sim,enc_cos_sim
51224,51224,False,0.002244,False,0.009417,0.005461,False,1.356139,1.434295,1.089368,0.015165,0.620764,0.012215,IT only,0.781824,1.287962,0.303512,0.678796,-0.315993
14538,14538,False,0.004970,False,0.008652,0.006621,False,2.033081,2.228742,1.756093,0.010553,0.693029,0.007614,IT only,0.925675,1.212323,0.381777,0.538052,-0.430217
60186,60186,False,0.009121,False,0.007596,0.008437,False,1.697013,1.558748,1.790660,0.014414,0.650730,0.011075,IT only,0.738039,1.112684,0.331648,0.714217,-0.556696
3792,3792,False,0.002327,False,0.007561,0.004674,False,2.004497,2.098311,1.756544,0.019425,0.744636,0.013043,IT only,0.860620,1.198700,0.358980,0.721904,-0.359496
21710,21710,False,0.008610,False,0.007376,0.008057,False,1.645800,1.688388,1.616128,0.012726,0.630944,0.010085,IT only,0.861562,1.136083,0.379181,0.702146,-0.371451
24731,24731,False,0.007945,False,0.007158,0.007592,False,1.914850,1.937446,1.898292,0.014258,0.655771,0.010871,IT only,0.954068,1.188913,0.401235,0.724845,-0.503147
4940,4940,False,0.005053,False,0.005609,0.005302,False,1.772930,1.780608,1.765998,0.015082,0.702384,0.010736,IT only,1.004360,1.340401,0.374649,0.628947,-0.418253
62571,62571,False,0.008807,False,0.004892,0.007051,False,1.622470,1.483220,1.685378,0.019504,0.614624,0.015866,IT only,0.711940,1.189054,0.299372,0.808625,-0.551995
1746,1746,False,0.006107,False,0.004677,0.005466,False,1.541892,1.618612,1.494107,0.012829,0.657272,0.009760,IT only,0.839238,1.219396,0.344121,0.622194,-0.353978
11743,11743,False,0.007542,False,0.004623,0.006233,False,1.735984,1.816740,1.695722,0.013883,0.716744,0.009685,IT only,0.840073,1.131233,0.371309,0.678448,-0.370328


## Offline dashboard

In [3]:
from pathlib import Path
import torch as th
load_path = Path(
    "/dlabscratch1/cdumas/representation-structure-comparison/results-runai/max_activating_examples"
)
save_path = Path(
    "/dlabscratch1/cdumas/representation-structure-comparison/results/max_activating_examples"
)
# generate the max activating examples
# max_activating_ex_mini = th.load(save_path / "max_activating_examples_mini_final_cleaned.pt")
# max_activating_examples_base = th.load(save_path / "max_activating_examples_base.pt")
max_activating_examples_chat = th.load(load_path / "chat/chat_final.pt")

/tmp/ipykernel_924170/3513962368.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  max_activating_examples_chat = th.load(load_path / "chat/chat_final.pt")


In [4]:
max_activating_examples_base = th.load(load_path / "base/base_final.pt")


/tmp/ipykernel_924170/2120046621.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  max_activating_examples_base = th.load(load_path / "base/base_final.pt")


In [5]:
from feature_dashboard import FeatureCentricDashboard
from transformers import AutoTokenizer
dashboard = FeatureCentricDashboard(max_activation_examples=max_activating_examples_chat, tokenizer=AutoTokenizer.from_pretrained(instruct_model))
dashboard.display()

In [6]:
dashboard_base = FeatureCentricDashboard(max_activation_examples=max_activating_examples_base, tokenizer=AutoTokenizer.from_pretrained(instruct_model))
dashboard_base.display()

## Online dashboard

In [1]:
# mistake feature: 72073
# sensitive feature: 51408

import sys
sys.path.append("..")
dispatch = True
crosscoder_device = "cpu"
base_device = "cuda:0"
instruct_device = "cuda:1"
base_model = "google/gemma-2-2b"
instruct_model = "google/gemma-2-2b-it"
layer = 13
crosscoder_path = "/dlabscratch1/jminder/repositories/representation-structure-comparison/checkpoints/l13-mu4.1e-02-lr1e-04/ae_final.pt"
from feature_dashboard import OnlineFeatureCentricDashboard
from nnterp import load_model
from dictionary_learning import CrossCoder
dtype = "float16"
b_model = load_model(
    base_model, attn_implementation="eager", torch_dtype=dtype, device_map=base_device, dispatch=dispatch
)
it_model = load_model(
    instruct_model, attn_implementation="eager", torch_dtype=dtype, device_map=instruct_device, dispatch=dispatch
)
crosscoder = CrossCoder.from_pretrained(crosscoder_path, device=crosscoder_device)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
dashboard = OnlineFeatureCentricDashboard(b_model, it_model, crosscoder, 13, crosscoder_device=crosscoder_device)
dashboard.display()

You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [10]:
# with it_model.generate("Hello, how are you?", max_new_tokens=12):
#     output = it_model.generator.output.save()
# print(it_model.tokenizer.decode(output[0]))
from datasets import load_dataset
dataset = load_dataset("truthful_qa", "generation")

In [23]:
from collections import Counter

# Assuming 'dataset' is already defined and contains the data
category_counts = Counter(dataset['validation']['category'])
category_counts_dict = dict(category_counts)
for category, count in category_counts_dict.items():
    print(f"{category}: {count}")

# print(list(set(dataset['validation']['category'])))
# # count Politics
# dataset['validation']['category'].count('Politics')



Misconceptions: 100
Proverbs: 18
Misquotations: 16
Conspiracies: 25
Superstitions: 22
Paranormal: 26
Fiction: 30
Myths and Fairytales: 21
Indexical Error: Identity: 9
Indexical Error: Other: 21
Indexical Error: Time: 16
Indexical Error: Location: 11
Distraction: 14
Subjective: 9
Advertising: 13
Religion: 15
Logical Falsehood: 14
Stereotypes: 24
Misconceptions: Topical: 4
Education: 10
Nutrition: 16
Health: 55
Psychology: 19
Sociology: 55
Economics: 31
Politics: 10
Law: 64
Science: 9
History: 24
Language: 21
Weather: 17
Confusion: People: 23
Confusion: Places: 15
Confusion: Other: 8
Finance: 9
Misinformation: 12
Statistics: 5
Mandela Effect: 6
